<h2>
An Analysis of <i>The Lazy Fundamental Analyst</i>
</h2>
<p>
The book <i>The Lazy Fundamental Analyst</i> by Fred Piard proposes an investment strategy that utilizes fundamental factors
like Price Earnings (PE) and Return on Assets (ROA), combined with a market timing signal based on S&P 500 earnings per share
momentum and unemployment momentum. This notebook attempts to verify some of these results.
</p>
<h3>
Market Timing Signals
</h3>
<h4>
EPS Estimate Momentum
</h4>
<blockquote>
<p>
"EPS Estimate Momentum" is an aggregate fundamental ratio over the S&P 500 Index. It gives a bearish signal when
the S&P 500 current year EPS estimate is below its own value three months previously, and a bullish signal when it is
above this value.
</p>
<p>
<i>The Lazy Fundamental Analyst</i> by Fred Paird
</p>
</blockquote>
<h4>
Unemployment Momentum
</h4>
<blockquote>
<p>
It [Unemployment Momentum] gives a bearish signal when the US enemployment rate is above it's own value three months ago,
and a bullish signal when it is below this value. The US unemployment rate is published once a month, usually in the first half
of the month for the previous month.
</p>
</blockquote>
<p>
When one of the indicators is "bullish", the portfolio is invested in stocks. When the two indicators are "bearish" the
portfolio is hedged with the ETF SH (ProShares 1x Short S&P 500).
</p>
<h3>
EPS Estimate Momentum
</h3>
<p>
Monthly S&P 500 earnings yield, by month, is published on the NASDAQ data site, along with several other S&P 500 statistics. See
<a href="https://data.nasdaq.com/data/MULTPL-sp-500-ratios">S&P 500 Ratios</a>. Fred Piard's EPS estimate momentum is the yearly
sum, so the estimate for 12 months must be summed for the yearly value.
</p>

In [ ]:
from datetime import datetime, timedelta
from typing import List, Callable

import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# https://data.nasdaq.com/tools/python
# pip install Nasdaq-Data-Link
import nasdaqdatalink as nasd
from pandas import DatetimeIndex


def convert_date(some_date):
    if type(some_date) == str:
        some_date = datetime.fromisoformat(some_date)
    elif type(some_date) == np.datetime64:
        ts = (some_date - np.datetime64('1970-01-01T00:00')) / np.timedelta64(1, 's')
        some_date = datetime.utcfromtimestamp(ts)
    return some_date


def findDateIndex(date_index: DatetimeIndex, search_date: datetime) -> int:
    '''
    In a DatetimeIndex, find the index of the date that is nearest to search_date.
    This date will either be equal to search_date or the next date that is less than
    search_date
    '''
    index: int = -1
    i = 0
    search_date = convert_date(search_date)
    date_t = datetime.today()
    for i in range(0, len(date_index)):
        date_t = convert_date(date_index[i])
        if date_t >= search_date:
            break
    if date_t > search_date:
        index = i - 1
    else:
        index = i
    return index


start_date_str = '2007-01-03'
start_date: datetime = datetime.fromisoformat(start_date_str)
# The "current date"
end_date: datetime = datetime.today() - timedelta(days=1)

# For all of the S&P ratios see https://data.nasdaq.com/data/MULTPL-sp-500-ratios
# https://data.nasdaq.com/data/MULTPL/SP500_EARNINGS_YIELD_MONTH-sp-500-earnings-yield-by-month
# Monthly EPS estimates from 1871(!) to present
s_and_p_eps_raw = nasd.get("MULTPL/SP500_EARNINGS_YIELD_MONTH")

eps_index = s_and_p_eps_raw.index
ix_start = findDateIndex(eps_index, start_date - timedelta(weeks=52))
s_and_p_eps = s_and_p_eps_raw[:][ix_start:]

s_and_p_eps.plot(grid=True, title="Monthly S&P 500 Earnings yield", figsize=(10, 6))

s_and_p_eps_yearly = s_and_p_eps.rolling(12).sum()
s_and_p_eps_yearly = s_and_p_eps_yearly[:][12:]

s_and_p_eps_yearly.plot(grid=True, title="Yearly S&P 500 Earnings yield", figsize=(10, 6))

<h3>
Unemployment Momentum
</h3>
<p>
Fred Piard writes that unemployment data "is widely broadcasted on fiancial and news websites. This is not much help for someone
who wants to perform quantitative analysis of the unemployment rate as a market signal.
</p>
<p>
United States unemployment rate statistics are published by the US Bureau of Labor Statistics. The data can be downloaded
via a REST interface. To do this, a data code is required. The data code for the seasonally adjusted unemployment rate is:
</p>
<pre>
Data ID: LNS14000000
HTTP Type:	GET
URL (JSON):	https://api.bls.gov/publicAPI/v2/timeseries/data/
</pre>
<p>
There is a Python library for accessing the BLS data: <a href="https://pypi.org/project/bls-data/">PyBLS</a>. The BLS web site
also publishes some code that does not use the PyBLS API. See
<a href="https://www.bls.gov/developers/api_python.htm">Accessing the Public Data with Python</a>
</p>